## 聊天模型
接收一系列消息并返回消息输出的模型

In [1]:
import json
import os
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage 

# 读取配置文件
with open("config.json", "r") as f:
    config = json.load(f)
    api_key = config["api_key"]

# 设置环境变量
os.environ["OPENAI_API_KEY"] = config["api_key"]
os.environ["OPENAI_API_BASE"] = config["api_base"]

# 创建 OpenAI 客户端
llm = ChatOpenAI(model="deepseek-v3")

# 调用聊天模型
r = llm.invoke(
    [
    SystemMessage(content="你是一个AI助手，请回答用户的问题。"),
    HumanMessage(content="我喜欢吃什么？"),
    ]
)

print(r)


content='作为一个AI，我无法直接了解你的个人喜好。不过，你可以告诉我你平时喜欢吃什么类型的食物，我可以根据你的口味推荐一些美食或食谱！比如，你喜欢甜食、辣味、还是清淡的食物？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 16, 'total_tokens': 61, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-v3', 'system_fingerprint': None, 'id': 'chatcmpl-e51feae6-884f-96f4-91fe-90bf83a8a461', 'finish_reason': 'stop', 'logprobs': None} id='run-b4491ef1-ee34-4d1e-ac4b-e57e93e1d7e3-0' usage_metadata={'input_tokens': 16, 'output_tokens': 45, 'total_tokens': 61, 'input_token_details': {}, 'output_token_details': {}}


## 函数调用模型
可以提供结构化的数据输出

In [12]:
import json
import os
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from pydantic import BaseModel, Field

# 读取配置文件
with open("config.json", "r") as f:
    config = json.load(f)
    api_key = config["api_key"]

# 设置环境变量
os.environ["OPENAI_API_KEY"] = config["api_key"]
os.environ["OPENAI_API_BASE"] = config["api_base"]

# 创建 OpenAI 客户端
llm = ChatOpenAI(model= config["deepseek_model"])

class WeatherTool(BaseModel):
    """天气查询工具"""
    
    city: str = Field(description="城市名称")
    info_type: str = Field(
        default="all",
        description="查询信息类型 (temperature/condition/all)"
    )

def get_weather_info(city: str, info_type: str = "all") -> dict:
    """模拟天气 API 调用"""
    weather_data = {
        "深圳": {
            "temperature": 25,
            "condition": "晴朗",
            "humidity": 65
        }
    }
    
    if city not in weather_data:
        return {"error": f"未找到 {city} 的天气信息"}
        
    data = weather_data[city]
    if info_type == "temperature":
        return {"temperature": data["temperature"]}
    elif info_type == "condition":
        return {"condition": data["condition"]}
    return data

class WeatherAssistant:
    def __init__(self, api_key: str,openai_api_base:str,model_name:str):
        self.llm = ChatOpenAI(
            openai_api_key=api_key,
            openai_api_base=openai_api_base,
            model_name=model_name
        )
        
        # 绑定工具
        self.llm_with_tools = self.llm.bind_tools(
            tools=[WeatherTool],
            tool_choice="auto",
            strict=True
        )
        
    def query_weather(self, user_input: str):
        try:
            # 系统提示
            messages = [
                SystemMessage(content="你是一个专业的天气助手，可以帮助用户查询天气信息。"),
                HumanMessage(content=user_input)
            ]
            
            # 调用模型
            response = self.llm_with_tools.invoke(messages)
            
            # 处理工具调用
            if hasattr(response, 'tool_calls') and response.tool_calls:
                results = []
                for tool_call in response.tool_calls:
                    args = json.loads(tool_call['args'])
                    weather_info = get_weather_info(**args)
                    results.append(weather_info)
                
                # 让模型解释结果
                messages.append(response)
                messages.append(
                    SystemMessage(content=f"天气数据: {json.dumps(results, ensure_ascii=False)}")
                )
                final_response = self.llm.invoke(messages)
                return final_response.content
                
            return response.content
            
        except Exception as e:
            return f"查询出错: {str(e)}"

# 使用示例
if __name__ == "__main__":
    assistant = WeatherAssistant(config["api_key"],config["api_base"],config["deepseek_model"])
    
    # 测试查询
    queries = [
        "深圳今天天气怎么样？",
        "我需要带伞吗？",
        "现在温度多少度？"
    ]
    
    for query in queries:
        print(f"\n用户: {query}")
        response = assistant.query_weather(query)
        print(f"助手: {response}")



用户: 深圳今天天气怎么样？
助手: 查询出错: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': '<400> InternalError.Algo.InvalidParameter: The tool call is not supported.', 'type': 'invalid_request_error'}, 'id': 'chatcmpl-eee32a5d-16f1-956e-9bdd-22ca986fccc0', 'request_id': 'eee32a5d-16f1-956e-9bdd-22ca986fccc0'}

用户: 我需要带伞吗？
助手: 查询出错: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': '<400> InternalError.Algo.InvalidParameter: The tool call is not supported.', 'type': 'invalid_request_error'}, 'id': 'chatcmpl-b30398e5-6e08-9f60-abe6-278caa1d84d4', 'request_id': 'b30398e5-6e08-9f60-abe6-278caa1d84d4'}

用户: 现在温度多少度？
助手: 查询出错: Error code: 400 - {'error': {'code': 'invalid_parameter_error', 'param': None, 'message': '<400> InternalError.Algo.InvalidParameter: The tool call is not supported.', 'type': 'invalid_request_error'}, 'id': 'chatcmpl-fdf34e76-9ee8-9ff6-91cb-2ab26e5eba61', 'request_id': 'fdf34e76-9ee8-9ff6-91cb-2ab26e5eb

## 文本嵌入模型
将文本转换为向量（一系列包含文本语义含义的数字）

In [24]:
import json
import os
from langchain_openai import OpenAIEmbeddings
import dashscope
from dashscope import TextEmbedding
import numpy as np

# 读取配置文件
with open("config.json", "r") as f:
    config = json.load(f)
    api_key = config["api_key"]

# 设置环境变量
os.environ["DASHSCOPE_API_KEY"] = config["api_key"]

class DashScopeEmbeddings:
    def __init__(self, api_key: str = None):
        """初始化 DashScope Embeddings"""
        self.api_key = api_key or os.getenv('DASHSCOPE_API_KEY')
        dashscope.api_key = self.api_key

    def embed_query(self, text: str) -> list:
        """获取单个文本的嵌入向量"""
        try:
            response = TextEmbedding.call(
                model='text-embedding-v2',  # 使用 text-embedding-v2 模型
                input=text
            )
            
            if response.status_code == 200:
                # 返回嵌入向量
                return response.output['embeddings'][0]
            else:
                raise Exception(f"API调用失败: {response.code} - {response.message}")
                
        except Exception as e:
            raise Exception(f"生成嵌入向量时出错: {str(e)}")

    def embed_documents(self, texts: list) -> list:
        """获取多个文本的嵌入向量"""
        try:
            response = TextEmbedding.call(
                model='text-embedding-v2',
                input=texts
            )
            
            if response.status_code == 200:
                # 返回嵌入向量列表
                return response.output['embeddings']
            else:
                raise Exception(f"API调用失败: {response.code} - {response.message}")
                
        except Exception as e:
            raise Exception(f"生成嵌入向量时出错: {str(e)}")

# 使用示例
def main():
    # 创建 embeddings 实例
    embeddings_model = DashScopeEmbeddings(api_key=config["api_key"])

    # 测试单个文本
    try:
        text = "我是一个特种兵，我正在执行任务"
        embedding = embeddings_model.embed_query(text)
        
        print(f"文本：{text}")
        print(f"向量前5个维度：{embedding["embedding"][:5]}")
        print(f"向量维度：{len(embedding["embedding"])}")
        
        # 测试多个文本
        texts = [
            "我是一个特种兵，我正在执行任务",
            "今天天气真好，适合出去玩",
            "人工智能正在改变世界"
        ]
        
        embeddings = embeddings_model.embed_documents(texts)
        print("\n多个文本的嵌入向量：")
        for i, (text, emb) in enumerate(zip(texts, embeddings)):
            print(f"\n文本 {i+1}：{text}")
            print(f"向量前5个维度：{emb["embedding"][:5]}")
            print(f"向量维度：{len(emb["embedding"])}")
            
    except Exception as e:
        print(f"错误：{str(e)}")

if __name__ == "__main__":
    main()


文本：我是一个特种兵，我正在执行任务
向量前5个维度：[0.005827370179008478, 0.019640165448568597, 0.005647014004844929, -0.028533590588357416, 0.034081097738491425]
向量维度：1536

多个文本的嵌入向量：

文本 1：我是一个特种兵，我正在执行任务
向量前5个维度：[0.005827370179008478, 0.019640165448568597, 0.005647014004844929, -0.028533590588357416, 0.034081097738491425]
向量维度：1536

文本 2：今天天气真好，适合出去玩
向量前5个维度：[-0.0017264320383616743, -0.023259233422935226, 0.059993727743405785, -0.02754057904414808, -0.03161609074126417]
向量维度：1536

文本 3：人工智能正在改变世界
向量前5个维度：[0.02374165535780939, -0.033151145116250044, -0.03279220000284904, -0.013242510790829968, 0.06194367099834502]
向量维度：1536
